In [1]:
from eis.vcf_dataloader import SplicingVCFDataloader
from eis import Eis, predict_all_table

Using TensorFlow backend.


In [2]:
from eis.utils import max_varEff

In [3]:
gtf = '../tests/data/test.gtf'
vcf = '../tests/data/test.vcf.gz'
fasta = '../tests/data/hg19.nochr.chr17.fa'
gtfIntervalTree = '../tests/data/test.pkl' # pickle exon interval Tree

In [4]:
dl = SplicingVCFDataloader(gtf, 
                          fasta,
                          vcf,
                          out_file=gtfIntervalTree,
                          split_seq=False)

In [ ]:
model = Eis(
    exon_cut_l=0,
    exon_cut_r=0,
    acceptor_intron_cut=6,
    donor_intron_cut=6,
    acceptor_intron_len=50,
    acceptor_exon_len=3,
    donor_exon_len=5,
    donor_intron_len=13)

In [ ]:
predictions = predict_all_table(model, dl, batch_size=1024, split_seq=False, assembly=False)

In [ ]:
predictions.head()

In [ ]:
predictionsMax = max_varEff(predictions)

In [ ]:
predictionsMax.head()